# <span style="color:green">4. Grille de score</span>

La grille de score créée regroupe les informations suivantes et attribue une note sur 1000 aux clients :

- Variables explicatives
- Classes des variables explicatives
- P-value associée au test de Wald pour chaque classe
- Note normalisée attribuée à chaque classe
- Contribution de la variable
- Taux de défaut observé de chaque classe
- Effectif de chaque classe

Les Règles d’acceptation sont : 

* **P-value associée à chaque classe** : toutes les classes doivent être significatives au seuil de 5 %.
  Le non-respect de cette règle induit l’existence d’une corrélation résiduelle entre classes et/ou variables.

* **Cohérence des signes** : le signe des coefficients doit être cohérent.

#### <span style="color:orange">Librairies</span>

In [1]:
import json
import pandas as pd
import plotly.figure_factory as ff

#### <span style="color:orange">Update des frontières de discretisation</span>

In [2]:
# Regroupements manuels éffectués
# Note : loan_intent est exclu car c'est une variable catégorielle (pas de "boundaries" numériques)
manual_mappings = {
    'person_age': {3: 2},
    'person_income': {3: 2, 4: 3, 5: 3},
    'cb_person_cred_hist_length': {2: 1}
}

file_path = '../result/1_discretisation/discretization_boundaries_final.json'
output_path = '../result/1_discretisation/discretization_boundaries_updated.json'

# Chargement du fichier existant
with open(file_path, 'r') as f:
    metadata_list = json.load(f)

# Application des mises à jour
for meta in metadata_list:
    variable = meta['variable']
   
    if variable in manual_mappings:
        mapping = manual_mappings[variable]
        old_boundaries = meta['boundaries']
        old_n_bins = meta['n_bins']
        
        # On reconstruit les indices originaux (ex: 0, 1, 2, 3...)
        # Si n_bins = 4, indices = [0, 1, 2, 3]
        original_indices = list(range(old_n_bins))
        
        # On calcule les nouveaux indices pour chaque bin original
        # Par défaut, i -> i. Si dans mapping, i -> mapping[i]
        new_indices = [mapping.get(i, i) for i in original_indices]
        
        # Filtrage des frontières (Boundaries)
        # Une frontière à l'index i sépare le bin i du bin i+1.
        # Si bin i et bin i+1 ont le même nouvel index, ils sont fusionnés -> On supprime la frontière.
        new_boundaries = []
        for i in range(len(old_boundaries)):
            # La frontière i sépare original_indices[i] et original_indices[i+1]
            bin_left = new_indices[i]
            bin_right = new_indices[i+1]
            
            if bin_left != bin_right:
                new_boundaries.append(old_boundaries[i])
        
        # Mise à jour de l'objet JSON
        meta['boundaries'] = new_boundaries
        meta['n_bins'] = len(new_boundaries) + 1
        meta['method'] += " + Regroupement Manuel"
        
        print(f"\n Variable '{variable}' mise à jour :")
        print(f"   - Anciens bins : {old_n_bins} -> Nouveaux : {meta['n_bins']}")
        print(f"   - Anciennes bornes : {old_boundaries}")
        print(f"   - Nouvelles bornes : {new_boundaries}")

# 4. Sauvegarde
# ---------------------------------------------------------
with open(output_path, 'w') as f:
    json.dump(metadata_list, f, indent=4)

print(f"\nFichier mis à jour sauvegardé sous : {output_path}")


 Variable 'person_income' mise à jour :
   - Anciens bins : 6 -> Nouveaux : 4
   - Anciennes bornes : [28590.0, 35000.0, 55000.0, 63000.0, 86000.0]
   - Nouvelles bornes : [28590.0, 35000.0, 63000.0]

 Variable 'person_age' mise à jour :
   - Anciens bins : 4 -> Nouveaux : 3
   - Anciennes bornes : [25.467301981083345, 36.729492133447906, 47.134892307254084]
   - Nouvelles bornes : [25.467301981083345, 36.729492133447906]

 Variable 'cb_person_cred_hist_length' mise à jour :
   - Anciens bins : 3 -> Nouveaux : 2
   - Anciennes bornes : [5.663768914271531, 11.723404344557768]
   - Nouvelles bornes : [5.663768914271531]

Fichier mis à jour sauvegardé sous : ../result/1_discretisation/discretization_boundaries_updated.json


#### <span style="color:orange">Chargement des résultats du modèle</span>

In [3]:
# DataFrame des données discrétisées
df = pd.read_csv("../datasets/processed/credit_risk_dataset_discretized_updated.csv")

In [5]:
# DataFrame du résumé du Logit_Stepwise
df_stepwise = pd.read_excel("../result/3_selection_variables/Logit_Stepwise_strict.xlsx", engine='openpyxl')

In [6]:
df_stepwise.head()

,Variable,Modalité,Coefficient,p-value,Significativité
0,loan_percent_income,loan_percent_income_0 (Référence),0.000000,_,_
1,loan_percent_income,loan_percent_income_1,0.325217,0.0,***
2,loan_percent_income,loan_percent_income_2,0.643083,0.0,***
3,loan_percent_income,loan_percent_income_3,2.786090,0,***
4,person_home_ownership,person_home_ownership_0 (Référence),0.000000,_,_


#### <span style="color:orange">Création de la grille de score</span>

In [7]:
import pandas as pd
import numpy as np
import json

def create_score_grid(logit_stepwise_path, df_discretized, y, boundaries_json_path):
    """
    Crée une grille de score à partir du fichier Logit_Stepwise.xlsx et ajoute les colonnes nécessaires.
    
    Parameters:
    -----------
    logit_stepwise_path : str
        Chemin vers le fichier Excel Logit_Stepwise.xlsx
    df_discretized : DataFrame
        DataFrame avec les variables discrétisées (après tous les traitements)
    y : Series
        Variable cible (loan_status)
    boundaries_json_path : str
        Chemin vers le fichier JSON contenant les bornes de discrétisation
    
    Returns:
    --------
    score_grid : DataFrame
        Grille de score complète
    """

    # Chargement du fichier Logit_Stepwise
    logit_df = pd.read_excel(logit_stepwise_path)
    
    # Chargement des bornes de discrétisation
    with open(boundaries_json_path, 'r') as f:
        boundaries_data = json.load(f) 
    boundaries_dict = {item['variable']: item['boundaries'] for item in boundaries_data}
    
    # Extraction des variables sélectionnées
    selected_variables = logit_df['Variable'].unique().tolist()

    # Ajout des colonnes manquantes
    rows = []  
    for _, row in logit_df.iterrows():
        var = row['Variable']
        modalite_str = row['Modalité']
        coef = row['Coefficient']        
        # Extraction du numéro de modalité 
        modalite_str = str(row['Modalité']) # Format: "variable_name_X" ou "variable_name_X (Référence)"        
        # Extraction du numéro après le dernier underscore
        clean_modalite = modalite_str.replace('(Référence)', '').replace('(Reférence)', '').strip()
        parts = clean_modalite.split('_')
        modalite_num = int(parts[-1])  # Ex: "loan_percent_income_0" -> 0 
        # Vérification si c'est une référence
        is_reference = '(Référence)' in modalite_str or '(Reférence)' in modalite_str       
        # Construction des bornes
        if var in boundaries_dict:
            bounds = boundaries_dict[var] 
            modalities_count = len(df_discretized[var].unique())       
            if modalite_num == 0:
                bornes = f"]-∞, {bounds[0]:.2f}]"
            elif modalite_num == modalities_count - 1:
                bornes = f"]{bounds[-1]:.2f}, +∞["
            else:
                if modalite_num - 1 < len(bounds) and modalite_num < len(bounds):
                    bornes = f"]{bounds[modalite_num-1]:.2f}, {bounds[modalite_num]:.2f}]"
                else:
                    bornes = f"Modalité {modalite_num}"
        else:
            bornes = f"Modalité {modalite_num}"
        
        # Calcule de l'effectif
        mask = (df_discretized[var] == modalite_num)
        effectif = mask.sum()
        
        # Calculer du taux de défaut observé
        if effectif > 0:
            taux_defaut = y[mask].mean()
        else:
            taux_defaut = np.nan
        
        rows.append({
            'Variable': var,
            'Modalité': f"{var}_{modalite_num}" + (" (Référence)" if is_reference else ""),
            'Bornes modalité': bornes,
            'Effectif': effectif,
            'Coefficient': coef,
            'p-value': row['p-value'],
            'Significativité': row['Significativité'],
            'Taux de défaut observé': taux_defaut,
            'modalite_num': modalite_num
        })
    
    grid = pd.DataFrame(rows)
    
    # ========================================================================
    # Calcul des Notes selon la formule
    # ========================================================================
    # Pour chaque variable, calculer max et min des coefficients
    var_stats = {}
    for var in selected_variables:
        var_data = grid[grid['Variable'] == var]
        beta_max = var_data['Coefficient'].max()
        beta_min = var_data['Coefficient'].min()
        p = len(var_data)
        var_stats[var] = {'beta_max': beta_max, 'beta_min': beta_min, 'p': p}
    
    # Nombre de variables
    #k = len(selected_variables)
    
    # Calcul du dénominateur global : somme des amplitudes (max - min) pour chaque variable
    # Formule: Σ(i=1 à k) [max(β_i) - min(β_i)]
    sum_amplitudes = sum([
        var_stats[var]['beta_max'] - var_stats[var]['beta_min']
        for var in selected_variables
    ])
    
    # Calcul de la note pour chaque modalité
    # Formule: N_i^j = |max(β_i) - β_i^j| / Σ[max(β_i) - min(β_i)] * 1000
    def calculate_note(row):
        var = row['Variable']
        beta_j = row['Coefficient']
        #beta_max = var_stats[var]['beta_max']
        beta_min = var_stats[var]['beta_min']
        
        numerator = abs(beta_min - beta_j)
        
        if sum_amplitudes != 0:
            note = (numerator / sum_amplitudes) * 1000
        else:
            note = 0
        
        return note
    
    grid['Note'] = grid.apply(calculate_note, axis=1)
    
    # ========================================================================
    # Calcul des Contributions selon la formule
    # ========================================================================
    total_population = len(df_discretized)
    
    # Calcul de r_j (part de la population) et note moyenne par variable
    note_means = {}
    for var in selected_variables:
        var_data = grid[grid['Variable'] == var]
        # r_j pour chaque modalité
        r_j = var_data['Effectif'] / total_population
        # Note moyenne pondérée: Σ(r_j * N_i^j)
        note_mean = (var_data['Note'] * r_j).sum()
        note_means[var] = note_mean
    
    # Calcul des numérateurs pour chaque variable
    # num = √[Σ(r_j * (N_i^j - N̄_i)²)]
    numerators = {}
    for var in selected_variables:
        var_data = grid[grid['Variable'] == var]
        note_mean = note_means[var]
        r_j = var_data['Effectif'] / total_population
        
        # Somme des (N_i^j - N̄_i)² pondérée par r_j
        var_sum = ((var_data['Note'] - note_mean) ** 2 * r_j).sum()
        # Prendre la racine carrée
        num = np.sqrt(var_sum) if var_sum > 0 else 0
        numerators[var] = num
    
    # Calcul du dénominateur : somme de tous les numérateurs
    # denom = Σ(pour toutes variables) √[Σ(r_j * (N_i^j - N̄_i)²)]
    denom = sum(numerators.values())
    
    # Calcul de la contribution pour chaque variable
    # Formule: c_i = num / denom
    contributions = {}
    for var in selected_variables:
        num = numerators[var]
        
        if denom > 0:
            contribution = num / denom
        else:
            contribution = 0
        
        contributions[var] = contribution
    
    # Appliquer la contribution (même valeur pour toutes les modalités d'une variable)
    grid['Contribution'] = grid['Variable'].map(contributions)

    # Finalisation et formattage de la grille
    score_grid = grid[[
        'Variable', 'Modalité', 'Bornes modalité', 'Effectif', 
        'Coefficient', 'Note', 'Contribution', 'Taux de défaut observé',
        'p-value', 'Significativité'
    ]].copy()
   
    # Renommer les colonnes pour les pourcentages
    score_grid = score_grid.rename(columns={
        'Contribution': 'Contribution (%)',
        'Taux de défaut observé': 'Taux de défaut observé (%)'
    })
    
    # Formattage des colonnes numériques
    score_grid['Coefficient'] = score_grid['Coefficient'].round(4)
    score_grid['Note'] = score_grid['Note'].round(2)
    score_grid['Contribution (%)'] = (score_grid['Contribution (%)'] * 100).round(2)
    score_grid['Taux de défaut observé (%)'] = (score_grid['Taux de défaut observé (%)'] * 100).round(2)
    
    return score_grid

In [13]:
# Génération de la grille de score finale
score_grid = create_score_grid(
    logit_stepwise_path='../result/3_selection_variables/Logit_Stepwise.xlsx',
    df_discretized=df,
    y=df["loan_status"],
    boundaries_json_path='../result/1_discretisation/discretization_boundaries_updated.json'
)

print("\n=== Grille de Score ===")
display(score_grid.style.hide(axis='index'))


=== Grille de Score ===


Variable,Modalité,Bornes modalité,Effectif,Coefficient,Note,Contribution (%),Taux de défaut observé (%),p-value,Significativité
loan_percent_income,loan_percent_income_0 (Référence),"]-∞, 0.15]",17185,0.000000,0.000000,31.410000,12.150000,_,_
loan_percent_income,loan_percent_income_1,"]0.15, 0.25]",9110,0.327600,36.040000,31.410000,18.620000,0.000000,***
loan_percent_income,loan_percent_income_2,"]0.25, 0.30]",2444,0.618900,68.110000,31.410000,25.700000,0.000000,***
loan_percent_income,loan_percent_income_3,"]0.30, +∞[",3833,2.946900,324.270000,31.410000,70.310000,0,***
person_home_ownership,person_home_ownership_0 (Référence),Modalité 0,2584,0.000000,0.000000,25.040000,7.470000,_,_
person_home_ownership,person_home_ownership_1,Modalité 1,13441,1.689800,185.940000,25.040000,12.570000,0.000000,***
person_home_ownership,person_home_ownership_2,Modalité 2,16547,2.595700,285.630000,25.040000,31.570000,0.000000,***
cb_person_default_on_file,cb_person_default_on_file_0 (Référence),Modalité 0,26829,0.000000,0.000000,15.310000,18.400000,_,_
cb_person_default_on_file,cb_person_default_on_file_1,Modalité 1,5743,1.175700,129.370000,15.310000,37.800000,0.000000,***
person_income,person_income_0,"]-∞, 28590.00]",3259,1.395900,153.610000,15.260000,47.410000,0.000000,***


In [14]:
# Export Excel
score_grid.to_excel(
    "../result/4_grille_de_score/grille_score.xlsx",
    index=False,
)

#### <span style="color:orange"> Grille de score pour les clients </span>

In [15]:
# Renommage ID si nécessaire
df['Client_ID'] = df.index

# Préparation du Dictionnaire de Scoring
# On transforme la grille en un dictionnaire {Variable: {Modalité_Index: Note}}
# Ex: {'loan_percent_income': {0: 0, 1: 38.04, 2: 68.11, 3: 324.27}}
score_map = {}
for _, row in score_grid.iterrows():
    var = row['Variable']
    raw_mod = row['Modalité']
    note = row['Note']
    # On extrait l'index numérique à la fin de la chaîne (ex: "loan_grade_0" -> 0)
    try:
        clean_mod = raw_mod.split(' (')[0] # Enlève "(Référence)"
        bin_idx = int(clean_mod.rsplit('_', 1)[1])       
        if var not in score_map:
            score_map[var] = {}
        score_map[var][bin_idx] = note
    except Exception:
        continue 

# Calcul des Scores Clients
df_scored = df.copy()
df_scored['Note_Finale'] = 0 # Initialisation

variables_du_modele = score_grid['Variable'].unique()
colonnes_notes = []

for var in variables_du_modele:
    if var in df_scored.columns:
        col_note = f"Note_{var}"
        colonnes_notes.append(col_note)       
        # Mapping : On remplace la valeur (0, 1, 2...) par sa Note correspondante
        df_scored[col_note] = df_scored[var].map(score_map.get(var, {})).fillna(0).astype(float)        
        # Ajout au total
        df_scored['Note_Finale'] += df_scored[col_note]

# Export Final
# Ajout de 'loan_status' et 'loan_grade' ici
output_cols = ['Client_ID'] + colonnes_notes + ['Note_Finale', 'loan_status', 'loan_grade']
df_scores = df_scored[output_cols]

display(df_scores.head(10))
df_scores.to_csv('../result/4_grille_de_score/scores_clients.csv', index=False)

,Client_ID,Note_loan_percent_income,Note_person_home_ownership,Note_cb_person_default_on_file,Note_person_income,Note_loan_intent,Note_Finale,loan_status,loan_grade
0,0,0.00,0.00,0.00,153.61,32.50,186.11,0,1
1,1,324.27,185.94,0.00,153.61,100.42,764.24,1,2
2,2,324.27,285.63,0.00,0.00,100.42,710.32,1,2
3,3,324.27,285.63,129.37,23.97,100.42,863.66,1,2
4,4,36.04,0.00,0.00,153.61,0.00,189.65,1,0
5,5,324.27,285.63,0.00,0.00,32.50,642.40,1,1
6,6,324.27,285.63,0.00,0.00,100.42,710.32,1,1
7,7,324.27,285.63,0.00,0.00,32.50,642.40,1,0
8,8,36.04,0.00,0.00,153.61,0.00,189.65,1,3
9,9,324.27,285.63,0.00,0.00,0.00,609.90,1,1


#### <span style="color:orange"> Performance des scores </span>

In [16]:
# Séparation des distributions
# Groupe "Non-Défaut" (0) et Groupe "Défaut" (1)
x0 = df_scores[df_scores['loan_status'] == 0]['Note_Finale']
x1 = df_scores[df_scores['loan_status'] == 1]['Note_Finale']

# 3. Création du graphique de densité (KDE Plot)
# Nous utilisons create_distplot de figure_factory qui est parfait pour ça
hist_data = [x0, x1]
group_labels = ['Non-Défaut (0)', 'Défaut (1)']
colors = ['#2ca02c', '#d62728']  # Vert pour Non-Défaut, Rouge pour Défaut

# Graphique
fig = ff.create_distplot(
    hist_data, 
    group_labels, 
    show_hist=False, 
    show_rug=False, 
    colors=colors,
    curve_type='kde' 
)

# 4. Mise en forme
fig.update_layout(
    title='<b>Distribution des Scores par Statut (Densité)</b>',
    xaxis_title='Score (Note Finale)',
    yaxis_title='Densité de Probabilité',
    template='plotly_white',
    legend=dict(x=0.8, y=0.9),
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Ajout d'une zone d'ombre sous les courbes pour l'esthétique
fig.update_traces(fill='tozeroy', selector=dict(type='scatter'))

fig.show()

Plus les distributions sont éloignées et plus le score est discriminant

#### <span style="color:orange"> Prochaine étape : La segmentation </span>